In [1]:
import sys
import warnings
import pandas as pd

# Add the directory to the system path
sys.path.append(r'C:\Users\n740789\Documents\sfdr_report_generator\python_scripts')

# Now you should be able to import your module
from html_table_generator import generate_html_table

In [2]:
# Set display options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Suppress the specific warning
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [3]:
def read_excel_table(file_path, sheet_name='Sheet1', skiprows=3):
    # First, read the entire Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skiprows)
    
    # Find the last valid row by looking for the first row where all values are NaN
    # or where the first column contains specific footer text indicators
    footer_indicators = ['Confidential', 'Powered by']
    
    last_valid_idx = None
    for idx, row in df.iterrows():
        # Check if all values in the row are NaN
        if row.isna().all():
            last_valid_idx = idx
            break
            
        # Check if the first column contains any footer indicators
        first_col_value = str(row.iloc[0])
        if any(indicator in first_col_value for indicator in footer_indicators):
            last_valid_idx = idx
            break
    
    # If we found a cutoff point, slice the dataframe
    if last_valid_idx is not None:
        df = df.iloc[:last_valid_idx]
    
    # Clean up any remaining NaN values
    df = df.dropna(how='all')
    
    return df

In [4]:
def process_excel_file(file_path):
    # Read the main dataframe and the two tables
    df = read_excel_table(file_path, sheet_name="Post-Contractual Info Data", skiprows=3)
    sector = read_excel_table(file_path, sheet_name="Sectorial Distribution", skiprows=3)
    investment = read_excel_table(file_path, sheet_name="Top Investments", skiprows=3)

    # Keep only the first 16 rows of investment table
    investment = investment.head(16)

    # Generate HTML tables without wrapper divs
    investment_html = generate_html_table(investment, "investment")
    sector_html = generate_html_table(sector, "sector")

    # Add the HTML strings as new columns to the main dataframe
    df['q03_t1'] = investment_html
    df['q04_t'] = sector_html

    return df

In [5]:
file_path = r"C:\Users\n740789\Documents\sfdr_report_generator\excel_books\aladdin_data\FIG05240 04-11-2024 Post-contractual Info.xlsx"
df = process_excel_file(file_path)

In [6]:
bbdd = pd.read_excel(r"C:\Users\n740789\Documents\sfdr_report_generator\excel_books\bbdd_sfdr_wip.xlsx")
bbdd.rename(columns={'aladdin_code':'security_description'}, inplace=True)

In [7]:
bbdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164 entries, 0 to 163
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   security_description  164 non-null    object
 1   {{product_name}}      164 non-null    object
 2   {{lei_code}}          107 non-null    object
 3   {{ref_period}}        126 non-null    object
 4   language              164 non-null    object
 5   narrative             149 non-null    object
 6   {{taxonomy_2022}}     60 non-null     object
 7   {{taxonomy_2023}}     101 non-null    object
 8   {{esg_score_2022}}    73 non-null     object
 9   {{esg_score_2023}}    99 non-null     object
dtypes: object(10)
memory usage: 12.9+ KB


In [8]:
bbdd.narrative.value_counts(normalize=True)[lambda x: x > 0.02].round(2)

narrative
art8_nois_nopai_noprev    0.23
art8_fi                   0.09
art8_fi_pt                0.07
art8_fi_eng               0.06
asg_fp                    0.05
asg_epsv                  0.05
art8_fp                   0.04
art8_fi_10_eng            0.03
art8_fi_10                0.03
art8_nopai_fp             0.02
art8_fp_pt                0.02
sostenible_fi             0.02
art8_nopai_epsv           0.02
Name: proportion, dtype: float64

In [9]:
# Set display options to show full content
pd.set_option("display.max_colwidth", None)

# Print the full HTML content
print("Full HTML content for q04_t:")
print(result_df["q04_t"].iloc[0])

# If you want to see both q03_t1 and q04_t
print("\nFull HTML content for q03_t1:")
print(result_df["q03_t1"].iloc[0])

Full HTML content for q04_t:


NameError: name 'result_df' is not defined

In [19]:
df.iloc[0,:10]

security_description             FIG05240
{{es_aligned}}                   0.969198
{{sust_invest}}                  0.683288
{{sust_invest_env}}              0.508448
{{sust_invest_soc}}              0.175085
{{esg_score_2024}}               64.92346
total_turnover_aligned           6.066225
total_turnover_nuclear           0.005651
total_turnover_gas               0.000657
total_turnover_nogasnonuclear    6.059917
Name: 0, dtype: object

In [22]:
bbdd.head()

,aladdin_code,{{product_name}},{{lei_code}},{{ref_period}},language,narrative,{{taxonomy_2022}},{{taxonomy_2023}},{{esg_score_2022}},{{esg_score_2023}}
0,EPH00107,Santander Sostenible RV Global PPSI,"95980020140005598726. Este Plan de Previsión está integrado en la Entidad de Previsión Social Voluntaria SANTANDER PREVISION I, EPSV INDIVIDUAL inscrita en el Registro de EPSVs del Gobierno Vasco con el número 229-B.",1/01/2024 - 31/12/2024,SP,sostenible_eq_epsv,"1.14% en 2022,",1.89% en 2023,A-,A-
1,EPV00005,"Santander Renta Fija, PPSI","95980020140005598823.Este Plan de Previsión está integrado en la Entidad de Previsión Social Voluntaria SANTANDER PREVISION I, EPSV INDIVIDUAL inscrita en el Registro de EPSVs del Gobierno Vasco con el número 229-B.",1/01/2024 - 31/12/2024,SP,art8_epsv,"0% en 2022,",0.39% en 2023,A-,A-
2,EPV00043,"Santander ASG Renta Variable Europa, PPSI","95980020140005600084. Este Plan de Previsión está integrado en la Entidad de Previsión Social Voluntaria SANTANDER PREVISION I, EPSV INDIVIDUAL inscrita en el Registro de EPSVs del Gobierno Vasco con el número 229-B",1/01/2024 - 31/12/2024,SP,asg_epsv,"1.27% en 2022,",3.39% en 2023,A-,A-
3,EPV00050,"Mi Plan Santander Decidido, PPSI","95980020140005600763. Este Plan de Previsión está integrado en la Entidad de Previsión Social Voluntaria SANTANDER PREVISION I, EPSV INDIVIDUAL inscrita en el Registro de EPSVs del Gobierno Vasco con el número 229-B.",1/01/2024 - 31/12/2024,SP,art8_nopai_epsv,NaN,1.77% en 2023,NaN,A-
4,EPV00051,"Mi Plan Santander Moderado, PPSI","95980020140005600860. Este Plan de Previsión está integrado en la Entidad de Previsión Social Voluntaria SANTANDER PREVISION I, EPSV INDIVIDUAL inscrita en el Registro de EPSVs del Gobierno Vasco con el número 229-B.",1/01/2024 - 31/12/2024,SP,art8_nopai_epsv,NaN,1.80% en 2023,NaN,A-


In [10]:
# left jon on security_description
result_df = pd.merge(df, bbdd, on='security_description', how='left')

In [11]:
# let's display resutl_df except columns "q03_t1" and "q04_t"
result_df.drop(columns=["q03_t1", "q04_t"])

,security_description,{{es_aligned}},{{sust_invest}},{{sust_invest_env}},{{sust_invest_soc}},{{esg_score_2024}},total_turnover_aligned,total_turnover_nuclear,total_turnover_gas,total_turnover_nogasnonuclear,total_turnover_aligned_exsovereign,total_turnover_nuclear_exsovereign,total_turnover_gas_exsovereign,total_turnover_nogasnonuclear_exsovereign,total_opex_aligned,total_opex_nuclear,total_opex_gas,total_opex_nogasnonuclear,total_opex_aligned_exsovereign,total_opex_nuclear_exsovereign,total_opex_gas_exsovereign,total_opex_nogasnonuclear_exsovereign,total_capex_aligned,total_capex_nuclear,total_capex_gas,total_capex_nogasnonuclear,total_capex_aligned_exsovereign,total_capex_nuclear_exsovereign,total_capex_gas_exsovereign,total_capex_nogasnonuclear_exsovereign,total_turnover_enabling,total_turnover_transition,total_opex_enabling,total_opex_transition,total_capex_enabling,total_capex_transition,portfolio_mv_exsov,{{product_name}},{{lei_code}},{{ref_period}},language,narrative,{{taxonomy_2022}},{{taxonomy_2023}},{{esg_score_2022}},{{esg_score_2023}}
0,FIG05240,0.969198,0.683288,0.508448,0.175085,64.92346,6.066225,0.005651,0.000657,6.059917,9.212582,0.008581,0.000998,9.203003,6.339804,0.003602,0.000209,6.335992,9.628058,0.005471,0.000318,9.622269,8.396524,0.002307,0.005851,8.388366,12.751533,0.003504,0.008885,12.739144,3.614374,0.215905,4.042979,0.047788,4.354931,0.189236,0.658472,"Santander Sostenible Crecimiento, FI",9598007UUGTJFT63KK09. ISIN: ES0107782007 (Clase A); ES0107782015 (Clase C); ES0107782023 (Clase I),1/01/2024 - 31/12/2024,SP,sostenible_fi,"2.45% en 2022,",4.87% en 2023,A-,A-


In [12]:
# sort result_df column names first securtiy_description then all other columns alphabetically
result_df.columns = sorted(result_df.columns, key=lambda x: (x != "security_description", x))

In [13]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 48 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   security_description                       1 non-null      object 
 1   language                                   1 non-null      float64
 2   narrative                                  1 non-null      float64
 3   portfolio_mv_exsov                         1 non-null      float64
 4   q03_t1                                     1 non-null      float64
 5   q04_t                                      1 non-null      float64
 6   total_capex_aligned                        1 non-null      float64
 7   total_capex_aligned_exsovereign            1 non-null      float64
 8   total_capex_enabling                       1 non-null      float64
 9   total_capex_gas                            1 non-null      float64
 10  total_capex_gas_exsovereign   

In [ ]:
# We need to multiply the values in the "{{es_aligned}}", "{{sust_invest}}", "{{sust_invest_env}}", and "{{sust_invest_soc}}" column by 100
result_df["{{es_aligned}}"] = result_df["{{es_aligned}}"] * 100
result_df["{{sust_invest}}"] = result_df["{{sust_invest}}"] * 100
result_df["{{sust_invest_env}}"] = result_df["{{sust_invest_env}}"] * 100
result_df["{{sust_invest_soc}}"] = result_df["{{sust_invest_soc}}"] * 100

# We need to calculate some new values for the SFDR report

# add new colum {{other_nones}} equal to 100 - {{es_aligned}}
result_df["{{other_nones}}"] = 100 - result_df["{{es_aligned}}"]
# add new colum {{other_non_sust}} equal to 100 - {{sust_invest}}
result_df["{{other_non_sust}}"] = 100 - result_df["{{sust_invest}}"]


# add new coulumn to result_df named "rest_capex_aligned" that's eqaul to 1 - total_capex_aligned
result_df["rest_capex_aligned"] = 100 - result_df["total_capex_aligned"]  
# add new coulumn to result_df named "rest_opex_aligned" that's eqaul to 1 - total_opex_aligned
result_df["rest_opex_aligned"] = 100 - result_df["total_opex_aligned"]
# add new coulumn to result_df named "rest_turnover_aligned" that's eqaul to 1 - total_turnover_aligned
result_df["rest_turnover_aligned"] = 100 - result_df["total_turnover_aligned"]
# add new coulumn to result_df named "rest_capex_aligned_exsovereign" that's eqaul to 1 - total_capex_aligned_exsovereign
result_df["rest_capex_aligned_exsovereign"] = 100 - result_df["total_capex_aligned_exsovereign"]
# add new coulumn to result_df named "rest_opex_aligned_exsovereign" that's eqaul to 1 - total_opex_aligned_exsovereign
result_df["rest_opex_aligned_exsovereign"] = 100 - result_df["total_opex_aligned_exsovereign"]
# add new coulumn to result_df named "rest_turnover_aligned_exsovereign" that's eqaul to 1 - total_turnover_aligned_exsovereign
result_df["rest_turnover_aligned_exsovereign"] = 100 - result_df["total_turnover_aligned_exsovereign"]